# Import Packages

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Import Raw Data

## Raw Data Fuel

In [2]:
fuel_raw = pd.read_excel('../data/processed/all_completed.xlsx')

fuel_raw.sample(10)

,code_unit,EGI,tipe,date,hour,SLOC,amount,Hm,Consumption,HM
733,EX41,EGI 37,day,2022-12-30,12:13:00,CN404,1176.0,28658.0,106.200000,10.0
2106,EX43,EGI 87,night,2022-11-07,00:17:00,CN404,986.0,26915.0,101.200000,10.0
655,EX41,EGI 37,day,2022-11-21,NaN,NaN,NaN,NaN,NaN,NaN
700,EX41,EGI 37,night,2022-12-13,00:11:00,CN403,991.0,28374.0,109.666667,9.0
2209,EX43,EGI 87,night,2022-12-28,00:37:00,CN404,1019.0,27842.0,103.700000,10.0
929,EX42,EGI 37,day,2022-04-05,12:00:00,CN404,960.0,25368.0,108.100000,10.0
383,EX41,EGI 37,night,2022-07-09,00:03:00,CN404,368.0,25718.0,134.400000,5.0
714,EX41,EGI 37,night,2022-12-20,00:25:00,CN404,1042.0,28490.0,131.000000,10.0
705,EX41,EGI 37,day,2022-12-16,11:59:00,CN403,395.0,28417.0,84.200000,5.0
1736,EX43,EGI 87,day,2022-05-09,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
fuel_interim = fuel_raw.groupby(['code_unit',
                                 'date']).agg({'amount':np.sum,
                                               'HM':np.sum,
                                               'Consumption':np.mean}).reset_index().copy()

In [4]:
fuel_interim.sample(10)

,code_unit,date,amount,HM,Consumption
367,EX42,2022-01-03,1833.0,18.0,120.110390
607,EX42,2022-08-31,2162.0,11.0,110.363636
883,EX43,2022-06-03,1751.0,20.0,87.393939
838,EX43,2022-04-19,1282.0,15.0,132.250000
579,EX42,2022-08-03,2102.0,18.0,149.708333
577,EX42,2022-08-01,2228.0,200.0,58.178947
1394,EX44,2022-10-27,1684.0,7.0,156.571429
1193,EX44,2022-04-09,1953.0,20.0,100.350000
797,EX43,2022-03-09,1859.0,20.0,98.792929
1104,EX44,2022-01-10,850.0,8.0,107.625000


## Raw Data Production

In [5]:
production = pd.read_excel('../data/processed/Produksi EX41 dan EX43.ods')

production.sample(10)

,date,Eq. Numb,EGI,HM,Production OB
585,2022-12-12,EX43,Excavator,20.0,8190.0
691,2023-02-03,EX43,Excavator,22.0,10277.0
494,2022-10-28,EX41,Excavator,22.0,9796.0
487,2022-10-24,EX43,Excavator,18.0,5918.0
342,2022-08-05,EX41,Excavator,16.0,7153.0
267,2022-06-25,EX43,Excavator,18.0,9237.0
605,2022-12-22,EX43,Excavator,12.0,3738.0
119,2022-03-02,EX43,Excavator,21.0,10089.0
535,2022-11-17,EX43,Excavator,19.0,8721.8
466,2022-10-14,EX41,Excavator,0.0,0.0


In [6]:
production_interim = production[['date',
                                 'Eq. Numb',
                                 'HM',
                                 'Production OB']].copy()

In [7]:
production_interim.rename(columns={"Eq. Numb": "code_unit"}, inplace=True)

In [8]:
production_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718 entries, 0 to 717
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           718 non-null    datetime64[ns]
 1   code_unit      718 non-null    object        
 2   HM             718 non-null    float64       
 3   Production OB  718 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 22.6+ KB


In [9]:
production_interim.sample(10)

,date,code_unit,HM,Production OB
136,2022-03-11,EX41,21.0,11163.0
313,2022-07-20,EX43,16.0,7612.5
308,2022-07-18,EX41,16.0,6442.0
219,2022-04-21,EX43,12.0,4972.0
102,2022-02-22,EX41,10.0,4620.0
461,2022-10-11,EX43,20.0,9503.0
384,2022-08-26,EX41,19.0,9316.0
168,2022-03-27,EX41,22.0,10076.0
408,2022-09-07,EX41,19.0,8118.0
354,2022-08-11,EX41,17.0,7482.0


In [10]:
production_interim['date'] = production_interim['date'].astype(str) 

## Raw Data Ritase

In [11]:
ritase_files = glob.glob('../data/processed/ritase_*.xls')

temp = []
for i in ritase_files:
      frame = pd.read_csv(i)
      frame['code_unit'] = i[25:29].upper()
      temp.append(frame)

ritase = pd.concat(temp, axis=0, ignore_index=True)

ritase['OB'] = ritase['ritase'] * 43

ritase

,date,ritase,code_unit,OB
0,2022-01-01,0,EX42,0
1,2022-01-02,135,EX42,5805
2,2022-01-03,149,EX42,6407
3,2022-01-04,159,EX42,6837
4,2022-01-05,135,EX42,5805
...,...,...,...,...
727,2022-12-28,211,EX44,9073
728,2022-12-29,164,EX44,7052
729,2022-12-30,255,EX44,10965
730,2022-12-31,192,EX44,8256


In [12]:
ritase_interim = ritase[['date', 'code_unit', 'OB']].copy()

In [13]:
ritase_interim.rename(columns={"OB": "Production OB"}, inplace=True)

In [14]:
ritase_interim.sample(10)

,date,code_unit,Production OB
623,2022-09-15,EX44,6751
154,2022-06-04,EX42,8643
383,2022-01-18,EX44,6407
565,2022-07-19,EX44,10062
403,2022-02-07,EX44,2666
462,2022-04-07,EX44,7138
438,2022-03-14,EX44,7181
248,2022-09-06,EX42,8170
559,2022-07-13,EX44,7783
254,2022-09-12,EX42,10191


In [15]:
ritase_interim.insert(2, 'HM', np.nan)

ritase_interim.sample(5)

,date,code_unit,HM,Production OB
343,2022-12-10,EX42,NaN,10191
574,2022-07-28,EX44,NaN,5891
365,2023-01-01,EX42,NaN,0
131,2022-05-12,EX42,NaN,4773
126,2022-05-07,EX42,NaN,5203


In [16]:
ritase_interim.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 0 to 731
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           732 non-null    object 
 1   code_unit      732 non-null    object 
 2   HM             0 non-null      float64
 3   Production OB  732 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 23.0+ KB


In [17]:
production_final = pd.concat([production_interim, ritase_interim], 
                             axis=0, ignore_index=True)

production_final.sample(5)

,date,code_unit,HM,Production OB
356,2022-08-12,EX41,21.0,9589.0
945,2022-08-16,EX42,NaN,4859.0
1069,2022-12-18,EX42,NaN,12384.0
1074,2022-12-23,EX42,NaN,11008.0
1101,2022-01-18,EX44,NaN,6407.0


In [18]:
production_final

,date,code_unit,HM,Production OB
0,2022-01-02,EX41,20.0,10350.0
1,2022-01-02,EX43,21.0,10800.0
2,2022-01-03,EX41,22.0,11565.0
3,2022-01-03,EX43,21.0,9675.0
4,2022-01-04,EX41,14.0,6750.0
...,...,...,...,...
1445,2022-12-28,EX44,NaN,9073.0
1446,2022-12-29,EX44,NaN,7052.0
1447,2022-12-30,EX44,NaN,10965.0
1448,2022-12-31,EX44,NaN,8256.0


## Finalize Data

In [19]:
fuel_interim.sample(5)

,code_unit,date,amount,HM,Consumption
896,EX43,2022-06-16,1721.0,19.0,93.022727
1195,EX44,2022-04-11,1881.0,21.0,89.827273
1000,EX43,2022-09-28,2074.0,23.0,91.359848
1201,EX44,2022-04-17,1914.0,21.0,90.100000
702,EX42,2022-12-04,0.0,0.0,NaN


In [20]:
production_interim.sample(5)

,date,code_unit,HM,Production OB
597,2022-12-18,EX43,21.0,9846.0
169,2022-03-27,EX43,21.0,9936.0
379,2022-08-23,EX43,15.0,7110.0
39,2022-01-21,EX43,14.0,6138.0
207,2022-04-15,EX43,7.0,2728.0


In [21]:
df = pd.merge(fuel_interim, production_final, on=["date", "code_unit"],
              how='left')

In [22]:
df.columns = df.columns.str.upper() 

In [23]:
df.sample(5)

,CODE_UNIT,DATE,AMOUNT,HM_X,CONSUMPTION,HM_Y,PRODUCTION OB
858,EX43,2022-05-09,1431.0,4.0,208.500000,NaN,NaN
128,EX41,2022-05-09,716.0,8.0,89.500000,NaN,NaN
909,EX43,2022-06-29,2007.0,23.0,79.640152,23.0,11790.0
494,EX42,2022-05-10,2201.0,19.0,125.300000,NaN,8901.0
1098,EX44,2022-01-04,1785.0,20.0,118.439560,NaN,9589.0


In [24]:
df.rename(columns={'HM_X': 'HM_CALC',
                   'HM_Y': 'HM_HRS',
                   'PRODUCTION OB' : "PRODUCTION_OB"}, inplace=True)

df.sample(5)

,CODE_UNIT,DATE,AMOUNT,HM_CALC,CONSUMPTION,HM_HRS,PRODUCTION_OB
1310,EX44,2022-08-04,1863.0,22.0,88.136364,NaN,10363.0
975,EX43,2022-09-03,1525.0,15.0,129.050000,20.0,8481.0
1263,EX44,2022-06-18,1754.0,18.0,96.712500,NaN,11610.0
836,EX43,2022-04-17,1860.0,22.0,84.363636,13.0,6021.0
1403,EX44,2022-11-05,2012.0,16.0,136.516667,NaN,6880.0


In [25]:
df['CONSUMPTION_CALC'] = df['AMOUNT']/df['HM_CALC']

In [26]:
df['CONSUMPTION_HRS'] = df['AMOUNT']/df['HM_HRS']

In [27]:
df.sample(5)

,CODE_UNIT,DATE,AMOUNT,HM_CALC,CONSUMPTION,HM_HRS,PRODUCTION_OB,CONSUMPTION_CALC,CONSUMPTION_HRS
1065,EX43,2022-12-02,1959.0,21.0,92.872727,21.0,10922.0,93.285714,93.285714
668,EX42,2022-10-31,2163.0,17.0,139.778571,NaN,10449.0,127.235294,NaN
547,EX42,2022-07-02,2435.0,20.0,122.300000,NaN,3225.0,121.750000,NaN
110,EX41,2022-04-21,0.0,0.0,NaN,1.0,125.0,NaN,0.000000
324,EX41,2022-11-21,0.0,0.0,NaN,10.0,4055.2,NaN,0.000000


# Modeling

In [28]:
df_modeling = df[['CODE_UNIT','CONSUMPTION_HRS', 'HM_HRS', 'PRODUCTION_OB']].copy()

In [29]:
df_modeling.dropna(inplace=True)

In [30]:
df_modeling

,CODE_UNIT,CONSUMPTION_HRS,HM_HRS,PRODUCTION_OB
1,EX41,118.100000,20.0,10350.0
2,EX41,89.136364,22.0,11565.0
3,EX41,139.285714,14.0,6750.0
4,EX41,83.047619,21.0,11475.0
5,EX41,81.538462,13.0,6255.0
...,...,...,...,...
1090,EX43,79.444444,18.0,9051.0
1091,EX43,104.300000,20.0,9423.0
1092,EX43,76.523810,21.0,9436.0
1093,EX43,113.764706,17.0,8568.0


In [31]:
df_modeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 623 entries, 1 to 1094
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CODE_UNIT        623 non-null    object 
 1   CONSUMPTION_HRS  623 non-null    float64
 2   HM_HRS           623 non-null    float64
 3   PRODUCTION_OB    623 non-null    float64
dtypes: float64(3), object(1)
memory usage: 24.3+ KB


In [32]:
df_modeling.replace([np.inf, -np.inf], np.nan, inplace=True)

In [33]:
df_modeling.dropna(inplace=True)

In [34]:
df_modeling['CODE_UNIT'].value_counts()

EX43    305
EX41    293
Name: CODE_UNIT, dtype: int64

In [52]:
df_modeling.corr(numeric_only=True)

,CONSUMPTION_HRS,HM_HRS,PRODUCTION_OB
CONSUMPTION_HRS,1.000000,-0.602815,-0.549747
HM_HRS,-0.602815,1.000000,0.916818
PRODUCTION_OB,-0.549747,0.916818,1.000000


In [36]:
X = df_modeling.drop('CONSUMPTION_HRS', axis=1)
y = df_modeling['CONSUMPTION_HRS']

In [37]:
X = pd.get_dummies(X)

## Split Data

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
X_train

,HM_HRS,PRODUCTION_OB,CODE_UNIT_EX41,CODE_UNIT_EX43
199,5.0,2728.0,1,0
10,21.0,11520.0,1,0
816,17.0,7525.0,0,1
1017,12.0,4778.0,0,1
243,20.0,9005.0,1,0
...,...,...,...,...
78,13.0,5280.0,1,0
155,22.0,9912.0,1,0
341,22.0,9552.0,1,0
921,11.0,5468.0,0,1


## Preprocessing

In [40]:
scaler = StandardScaler()

In [41]:
numeric_feature = ['HM_HRS', 'PRODUCTION_OB']
categoric_feature = ['CODE_UNIT_EX41', 'CODE_UNIT_EX43']

scaler.fit(X_train[numeric_feature])

StandardScaler()

In [42]:
X_train_scaled = scaler.transform(X_train[numeric_feature])
X_test_scaled = scaler.transform(X_test[numeric_feature])

In [43]:
X_train_final = pd.concat([X_train[categoric_feature],
                           pd.DataFrame(X_train_scaled, index=X_train.index, columns=numeric_feature)], axis=1)

X_test_final = pd.concat([X_test[categoric_feature],
                           pd.DataFrame(X_test_scaled, index=X_test.index, columns=numeric_feature)], axis=1)

In [44]:
model = LinearRegression(n_jobs=-1)

In [45]:
X_train_final.sample(5)

,CODE_UNIT_EX41,CODE_UNIT_EX43,HM_HRS,PRODUCTION_OB
156,1,0,0.000454,0.104118
320,1,0,0.217235,-0.261358
1020,0,1,0.434016,0.034738
976,0,1,0.867578,0.422581
810,0,1,0.434016,-0.863406


## Train Model

In [46]:
model.fit(X_train_final,y_train)

LinearRegression(n_jobs=-1)

In [47]:
y_train_predict = model.predict(X_train_final)
y_test_predict = model.predict(X_test_final)

In [48]:
model.coef_

array([  9.64816499,  -9.64816499, -66.19590937,   4.33331057])

In [49]:
model.intercept_

113.84985342324481

## Evaluation

In [50]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [53]:
train_rmse = mean_squared_error(y_train, y_train_predict, squared=False)
test_rmse = mean_squared_error(y_test, y_test_predict, squared=False)
train_mae = mean_absolute_error(y_train, y_train_predict)
test_mae = mean_absolute_error(y_test, y_test_predict)
train_r2 = r2_score(y_train, y_train_predict)
test_r2 = r2_score(y_test, y_test_predict)

print("Linear Regression")
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)
print("Train MAE:", train_mae)
print("Test MAE:", test_mae)
print("Train R2", train_r2)
print("Test R2", test_r2)

Linear Regression
Train RMSE: 77.05739109071762
Test RMSE: 53.202904443970404
Train MAE: 40.129802777023706
Test MAE: 35.79165300378736
Train R2 0.3994794284347103
Test R2 -0.19922419676891345


# Export Dataset

In [55]:
df.to_csv("../data/processed/datamart_final.csv", index=False)